In [0]:
%sql
CREATE TABLE SOURCE_T (id INT, name VARCHAR(20), city VARCHAR(15));

In [0]:
%sql
INSERT INTO SOURCE_T 
(id, name, city)
VALUES 
(1, 'BHIM', 'Abad'), 
(2, 'RAJU', 'Surat'), 
(3, 'Jaggu', 'Bombay');

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM SOURCE_T;

id,name,city
1,BHIM,Abad
2,RAJU,Surat
3,Jaggu,Bombay


In [0]:
%sql
CREATE TABLE DEST_T (
  id INT, 
  name VARCHAR(20), 
  city VARCHAR(15), 
  is_active BOOLEAN, 
  insert_date DATE, 
  end_date DATE);

In [0]:
%sql
SELECT * FROM DEST_T;

id,name,city,is_active,insert_date,end_date


In [0]:
%sql
MERGE INTO DEST_T as d
USING SOURCE_T as s
  ON d.id = s.id
WHEN MATCHED THEN 
  UPDATE SET
    d.is_active = FALSE, 
    d.end_date = CURRENT_DATE;

INSERT INTO DEST_T 
  (id, name, city, is_active, insert_date, end_date)
SELECT 
  id, name, city, TRUE, CURRENT_DATE, NULL
FROM SOURCE_T;

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from DEST_T;

id,name,city,is_active,insert_date,end_date
2,RAJU,A,true,2025-03-10,null
4,Chutki,Surat,true,2025-03-10,null
2,RAJU,Surat,false,2025-03-10,2025-03-10
1,BHIM,Abad,true,2025-03-10,null
3,Jaggu,Bombay,true,2025-03-10,null


In [0]:
%sql
TRUNCATE TABLE SOURCE_T;
INSERT INTO SOURCE_T
(id, name, city)
VALUES (2, 'RAJU', 'A'),
       (4, 'Chutki', 'Surat');


num_affected_rows,num_inserted_rows
2,2


CHATGPT GENERATED SCD TYPE 2

```
%sql
MERGE INTO DEST_T d
USING (
    SELECT *, MD5(CONCAT_WS('|', col1, col2, col3, ..., col50)) AS hash_value
    FROM SOURCE_T
) s
ON d.id = s.id 
AND d.is_active = TRUE  -- Only compare active records

WHEN MATCHED AND d.hash_value <> s.hash_value THEN 
  UPDATE SET
    d.is_active = FALSE,
    d.end_date = CURRENT_DATE

WHEN MATCHED AND d.hash_value <> s.hash_value THEN 
  INSERT (id, col1, col2, ..., col50, is_active, insert_date, end_date, hash_value)
  VALUES (s.id, s.col1, s.col2, ..., s.col50, TRUE, CURRENT_DATE, NULL, s.hash_value)

WHEN NOT MATCHED THEN 
  INSERT (id, col1, col2, ..., col50, is_active, insert_date, end_date, hash_value)
  VALUES (s.id, s.col1, s.col2, ..., s.col50, TRUE, CURRENT_DATE, NULL, s.hash_value);
```